<a href="https://colab.research.google.com/github/rob-ec/colab-basic-gemini-chatbot/blob/main/basic_gemini_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot com Gemini AI

`@author` Robson Gomes <dev@robsongomes.me>

## Instalação de dependências e Importaçãoes

### Instalações

In [ ]:
!pip install -U -q google-generativeai

### Importações

In [ ]:
import google.generativeai as genai
import json

from IPython.display import display, Markdown

## Configurações

### Chave API

In [44]:
# @title Definindo Chave API {display-mode: "form"}
API_KEY = "COLE_AQUI_SUA_CHAVE_DA_API" #@param {type: "string"}

#### Implementando chave

In [ ]:
genai.configure(api_key = API_KEY)

### Funções auxiliares

In [ ]:
def displayObjectsAsTable(objects):
  keys = objects[0].__dict__.keys()
  rows = []

  for obj in objects:
    row = f"| {'|'.join(str(getattr(obj, key)) for key in keys)} |"
    rows.append(row)

  header = ' | '.join(keys)
  tableRows = "\n".join(rows)
  table = f"|{header}|\n|{'--|'*len(keys)}\n{tableRows}"

  display(Markdown(table))

In [ ]:
def displayAsJSON(dict):
  display(Markdown("```JSON\n" + json.dumps(dict, indent=4) + "\n```"))

In [ ]:
def getModelsNames(models):
  return [model.name[7:] for model in models]

In [ ]:
def displayModels(models, headers = ["Título", "Nome"]):
  header = ' | '.join(headers)

  rows = []

  for model in models:
    rows.append(f"| {model.display_name} | `{model.name[7:]}` |")

  tableRows = "\n".join(rows)
  table = f"|{header}|\n|{'--|'*len(headers)}\n{tableRows}"

  display(Markdown(table))


### Modelos disponíveis

#### Modelos

In [ ]:
# @title { display-mode: "form" }
modelList = []

for model in genai.list_models():
  if 'generateContent' in model.supported_generation_methods:
    modelList.append(model)

displayObjectsAsTable(modelList)

|name | base_model_id | version | display_name | description | input_token_limit | output_token_limit | supported_generation_methods | temperature | top_p | top_k|
|--|--|--|--|--|--|--|--|--|--|--|
| models/gemini-1.0-pro||001|Gemini 1.0 Pro|The best model for scaling across a wide range of tasks|30720|2048|['generateContent', 'countTokens']|0.9|1.0|None |
| models/gemini-1.0-pro-001||001|Gemini 1.0 Pro 001 (Tuning)|The best model for scaling across a wide range of tasks. This is a stable model that supports tuning.|30720|2048|['generateContent', 'countTokens', 'createTunedModel']|0.9|1.0|None |
| models/gemini-1.0-pro-latest||001|Gemini 1.0 Pro Latest|The best model for scaling across a wide range of tasks. This is the latest model.|30720|2048|['generateContent', 'countTokens']|0.9|1.0|None |
| models/gemini-1.0-pro-vision-latest||001|Gemini 1.0 Pro Vision|The best image understanding model to handle a broad range of applications|12288|4096|['generateContent', 'countTokens']|0.4|1.0|32 |
| models/gemini-1.5-pro-latest||001|Gemini 1.5 Pro|Mid-size multimodal model that supports up to 1 million tokens|1048576|8192|['generateContent', 'countTokens']|1.0|0.95|None |
| models/gemini-pro||001|Gemini 1.0 Pro|The best model for scaling across a wide range of tasks|30720|2048|['generateContent', 'countTokens']|0.9|1.0|None |
| models/gemini-pro-vision||001|Gemini 1.0 Pro Vision|The best image understanding model to handle a broad range of applications|12288|4096|['generateContent', 'countTokens']|0.4|1.0|32 |

### Parâmetros de Configuração

In [ ]:
# @title Configuração de IA Generativa { display-mode: "form", run: "auto" }
# @markdown `generation_config`:

candidate_count = 1 #@param {type:"slider", min:1, max:10, step:1}
temperature = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}
top_k = 32 #@param {type:"slider", min:1, max:32, step:1}
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}

generation_config = {
    "candidate_count": candidate_count,
    "temperature": temperature,
    "top_k": top_k,
    "top_p": top_p,
}

In [ ]:
# @title Segurança { display-mode: "form", run: "auto" }
# @markdown `safety_config`:

harassment_content_policy = "BLOCK_MEDIUM_AND_ABOVE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE"]
hate_speech_content_policy = "BLOCK_MEDIUM_AND_ABOVE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE"]
sexually_explicit_content_policy = "BLOCK_MEDIUM_AND_ABOVE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE"]
dangerous_content_policy = "BLOCK_MEDIUM_AND_ABOVE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE"]

safety_config = {
    "HARASSMENT": harassment_content_policy,
    "HATE": hate_speech_content_policy,
    "SEXUAL": sexually_explicit_content_policy,
    "DANGEROUS": dangerous_content_policy,
}

### Selecionar Modelo

In [ ]:
# @title Modelos que suportam geração de conteúdo { display-mode: "form" }
displayModels(modelList)

|Título | Nome|
|--|--|
| Gemini 1.0 Pro | `gemini-1.0-pro` |
| Gemini 1.0 Pro 001 (Tuning) | `gemini-1.0-pro-001` |
| Gemini 1.0 Pro Latest | `gemini-1.0-pro-latest` |
| Gemini 1.0 Pro Vision | `gemini-1.0-pro-vision-latest` |
| Gemini 1.5 Pro | `gemini-1.5-pro-latest` |
| Gemini 1.0 Pro | `gemini-pro` |
| Gemini 1.0 Pro Vision | `gemini-pro-vision` |

In [ ]:
# @title Definir modelo {display-mode: "form", run: "auto"}

model_name = "gemini-1.0-pro" # @param ['gemini-1.0-pro','gemini-1.0-pro-001','gemini-1.0-pro-latest','gemini-1.0-pro-vision-latest','gemini-1.5-pro-latest','gemini-pro','gemini-pro-vision']

In [ ]:
# @title Implementação do modelo {display-mode: "form"}

model = genai.GenerativeModel(
    model_name = model_name,
    safety_settings = safety_config,
    generation_config = generation_config
)

displayAsJSON({
    'model_name': model_name,
    'safety_settings': safety_config,
    'generation_config': generation_config
})

```JSON
{
    "model_name": "gemini-1.0-pro",
    "safety_settings": {
        "HARASSMENT": "BLOCK_MEDIUM_AND_ABOVE",
        "HATE": "BLOCK_MEDIUM_AND_ABOVE",
        "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
        "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE"
    },
    "generation_config": {
        "candidate_count": 1,
        "temperature": 0.5,
        "top_k": 32,
        "top_p": 0.9
    }
}
```

## Utilização

In [ ]:
# @title Requisição simples {display-mode: "form"}
# @markdown Ao terminar de digitar use `Ctrl+Enter` ou `Command+Enter`

request_input = "Olá, que tipo de pergunta posso fazer para você?" # @param {type: "string"}
response = model.generate_content(request_input)

display(Markdown(response.text))

Você pode me fazer perguntas sobre uma ampla gama de tópicos, incluindo:

**Informações gerais:**
* Fatos e estatísticas sobre qualquer assunto
* Definições e explicações de conceitos
* Notícias e eventos atuais
* Informações sobre pessoas, lugares e coisas

**Ciência e tecnologia:**
* Princípios científicos e descobertas
* Avanços tecnológicos e inovações
* Questões ambientais e sustentabilidade
* Exploração espacial e astronomia

**História e cultura:**
* Eventos históricos e figuras importantes
* Culturas diferentes e suas tradições
* Arte, literatura e música
* Filosofia e religião

**Geografia e viagens:**
* Países, cidades e marcos geográficos
* Culturas e costumes de diferentes regiões
* Dicas de viagem e recomendações
* Informações sobre transporte e acomodação

**Saúde e bem-estar:**
* Condições de saúde e tratamentos
* Nutrição e fitness
* Saúde mental e emocional
* Cuidados pessoais e beleza

**Entretenimento e esportes:**
* Filmes, programas de TV e música
* Esportes e atletas
* Jogos e passatempos
* Celebridades e cultura pop

**Outros tópicos:**
* Negócios e finanças
* Educação e carreira
* Política e governo
* Idiomas e tradução
* Matemática e lógica

### Chat

In [ ]:
# @title {display-mode: "form"}

enable_automatic_function_calling = True # @param {type: "boolean"}

chat = model.start_chat(history = [], enable_automatic_function_calling = enable_automatic_function_calling)

In [45]:
# @title Conversar { display-mode: "form" }
# @markdown **Execute esse bloco para iniciar uma conversa** e use a palavra `fim` para **encerrar**

prompt = input("Aguardando prompt... -> ")

while prompt != "fim":
  chat_response = chat.send_message(prompt)
  print("\nIA: " + chat_response.text)
  prompt = input("\nEu: ")

Aguardando prompt... -> Fale o nomde cinco jogadores do corinthians

IA: 1. Cássio
2. Fagner
3. Gil
4. Renato Augusto
5. Róger Guedes

Eu: Qual deles é goleiro?

IA: Cássio

Eu: fim
